In [1]:
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import layers, models

Using TensorFlow backend.


In [ ]:
class Data:
    def __init__(self, max_features=20000, maxlen=80):
        # train, test 데이터를 나눠서 각각 입력(X)과 출력(y)을 로드합니다.
        (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

        # 입력 문장의 최대 길이(80)보다 클 경우 잘라내고, 작을 경우 앞에 padding(0)을 추가하여 길이를 맞춥니다.
        x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
        x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test


# 데이터의 갯수와 내용을 살펴봅니다.
data = Data()
print('train:', len(data.y_train), ', test:', len(data.y_test))
print('label:', data.y_train[20000:20005])
print('features:', data.x_train[20000:20005])